# Deep Learning with PyTorch

In [1]:
import torch
import numpy as np

In [2]:
a = torch.FloatTensor(3, 2)
print(a)

tensor([[4.8019e+30, 2.0700e-19],
        [2.6793e+20, 4.7420e+30],
        [2.3745e+23, 3.0357e+32]])


In [3]:
a.zero_()  # initialize tensor

tensor([[0., 0.],
        [0., 0.],
        [0., 0.]])

In [4]:
b = torch.FloatTensor([[1,2,3],[3,2,1]])

In [5]:
b

tensor([[1., 2., 3.],
        [3., 2., 1.]])

In [6]:
n = np.zeros(shape=(3,2))
c = torch.tensor(n)

In [7]:
c

tensor([[0., 0.],
        [0., 0.],
        [0., 0.]], dtype=torch.float64)

In [8]:
n = np.zeros(shape=(3,2), dtype=np.float32)
c = torch.tensor(n)

In [9]:
c

tensor([[0., 0.],
        [0., 0.],
        [0., 0.]])

In [10]:
c.dtype

torch.float32

In [11]:
s = b.sum()

In [12]:
s

tensor(12.)

In [13]:
s.item()

12.0

In [14]:
torch.tensor(1)

tensor(1)

In [15]:
b.dtype

torch.float32

In [16]:
cb = b.to('cuda')

In [17]:
cb

tensor([[1., 2., 3.],
        [3., 2., 1.]], device='cuda:0')

In [18]:
b+1

tensor([[2., 3., 4.],
        [4., 3., 2.]])

In [19]:
cb+1

tensor([[2., 3., 4.],
        [4., 3., 2.]], device='cuda:0')

## Gradients

In [20]:
v1 = torch.tensor([1.0, 1.0], requires_grad=True)
v2 = torch.tensor([2.0, 2.0])

In [21]:
v_sum= v1 + v2
v_res = (v_sum*2).sum()

In [22]:
v_res

tensor(12., grad_fn=<SumBackward0>)

In [23]:
v1.is_leaf, v2.is_leaf

(True, True)

In [24]:
v_sum.is_leaf, v_res.is_leaf

(False, False)

In [25]:
v1.requires_grad

True

In [26]:
v2.requires_grad

False

In [27]:
v_res.requires_grad

True

In [28]:
v_sum.requires_grad

True

In [29]:
v_res.backward() # calculate gradient of our graph

In [30]:
v1.grad

tensor([2., 2.])

In [31]:
print(v2.grad)

None


In [32]:
v_res.grad

/home/dongho/anaconda3/envs/DRLH/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the gradient for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations.
  """Entry point for launching an IPython kernel.


## NN building blocks

In [33]:
import torch.nn as nn
l = nn.Linear(2,5)
v = torch.FloatTensor([1,2])

In [34]:
l(v)

tensor([ 1.4305, -0.3858,  1.1154,  0.3309,  0.2852], grad_fn=<AddBackward0>)

In [35]:
s = nn.Sequential(
    nn.Linear(2,5),
    nn.ReLU(),
    nn.Linear(5, 20),
    nn.ReLU(),
    nn.Linear(20, 10),
    nn.Dropout(p=0.3),
    nn.Softmax(dim=1)
)

In [36]:
s

Sequential(
  (0): Linear(in_features=2, out_features=5, bias=True)
  (1): ReLU()
  (2): Linear(in_features=5, out_features=20, bias=True)
  (3): ReLU()
  (4): Linear(in_features=20, out_features=10, bias=True)
  (5): Dropout(p=0.3, inplace=False)
  (6): Softmax(dim=1)
)

In [37]:
s(torch.FloatTensor([[1,2]]))

tensor([[0.1113, 0.0999, 0.0765, 0.1350, 0.0855, 0.0946, 0.0823, 0.0953, 0.1118,
         0.1077]], grad_fn=<SoftmaxBackward>)

## Custom Layers

In [38]:
class OurModule(nn.Module):
    def __init__(self, num_inputs, num_classes, dropout_prob = 0.3):
        super(OurModule, self).__init__()
        self.pipe = nn.Sequential(
            nn.Linear(num_inputs, 5),
            nn.ReLU(),
            nn.Linear(5, 20),
            nn.ReLU(),
            nn.Linear(20, num_classes),
            nn.Dropout(p=dropout_prob),
            nn.Softmax(dim=1)
        ) # register module
        pass
    
    def forward(self, x):
        return self.pipe(x)

In [39]:
net = OurModule(num_inputs=2, num_classes=3)
v = torch.FloatTensor([[2,3]])
out = net(v)
print(net)
print(out)

OurModule(
  (pipe): Sequential(
    (0): Linear(in_features=2, out_features=5, bias=True)
    (1): ReLU()
    (2): Linear(in_features=5, out_features=20, bias=True)
    (3): ReLU()
    (4): Linear(in_features=20, out_features=3, bias=True)
    (5): Dropout(p=0.3, inplace=False)
    (6): Softmax(dim=1)
  )
)
tensor([[0.4216, 0.2826, 0.2958]], grad_fn=<SoftmaxBackward>)


## Monitoring with TensorBoard         

In [40]:
import math
from tensorboardX import SummaryWriter

In [41]:
writer = SummaryWriter()
funcs = {"sin": math.sin, "cos":math.cos, "tan":math.tan}

In [42]:
for angle in range(-360, 360):
    angle_rad= angle*math.pi / 180
    for name, fun in funcs.items():
        val= fun(angle_rad)
        writer.add_scalar(name, val, angle)
        
writer.close()

## GAN on Atari images

In [43]:
import random
import argparse
import cv2

import torch
import torch.nn as nn
import torch.optim as optim
from tensorboardX import SummaryWriter

import torchvision.utils as vutils

import gym
import gym.spaces

import numpy as np

log = gym.logger
log.set_level(gym.logger.INFO)

LATENT_VECTOR_SIZE = 100 # same as codings_size
DISCR_FILTERS = 64
GENER_FILTERS = 64
BATCH_SIZE = 16

# dimension input image will be rescaled
IMAGE_SIZE = 64

LEARNING_RATE = 0.0001
REPORT_EVERY_ITER = 100
SAVE_IMAGE_EVERY_ITER = 1000

In [44]:
class InputWrapper(gym.ObservationWrapper): # wrapper for vanilla env
    def __init__(self, *args):
        super(InputWrapper, self).__init__(*args) # env goes in here?
        assert isinstance(self.observation_space, gym.spaces.Box) # check whether is observation is box
        old_space = self.observation_space
        self.observation_space = gym.spaces.Box(
            self.observation(old_space.low),
            self.observation(old_space.high), # what?
            dtype = np.float32)
        pass
    
    def observation(self, observation):
        new_obs = cv2.resize(
            observation, (IMAGE_SIZE, IMAGE_SIZE))
        new_obs = np.moveaxis(new_obs, 2,0) # from (210,160,3) to (3, 160, 210)
        return new_obs.astype(np.float32)

In [45]:
class Discriminator(nn.Module): # nn.Module !!
    def __init__(self, input_shape): # takes input shape as argument
        super(Discriminator, self).__init__()
        # this pipe converges image into the single number
        self.conv_pipe = nn.Sequential(
            nn.Conv2d(in_channels=input_shape[0], out_channels=DISCR_FILTERS,
                      kernel_size=4, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=DISCR_FILTERS, out_channels=DISCR_FILTERS*2,
                      kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(DISCR_FILTERS*2), # batch norm 2d!
            nn.ReLU(),
            nn.Conv2d(in_channels=DISCR_FILTERS * 2, out_channels=DISCR_FILTERS * 4,
                      kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(DISCR_FILTERS * 4),
            nn.ReLU(),
            nn.Conv2d(in_channels=DISCR_FILTERS * 4, out_channels=DISCR_FILTERS * 8,
                      kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(DISCR_FILTERS * 8),
            nn.ReLU(),
            nn.Conv2d(in_channels=DISCR_FILTERS * 8, out_channels=1, # to one!
                      kernel_size=4, stride=1, padding=0),
            nn.Sigmoid()
        )

    def forward(self, x):
        conv_out = self.conv_pipe(x)
        return conv_out.view(-1, 1).squeeze(dim=1) # what does view do? -> same as reshape but only view

In [46]:

class Generator(nn.Module):
    def __init__(self, output_shape):
        super(Generator, self).__init__()
        # pipe deconvolves input vector into (3, 64, 64) image
        self.pipe = nn.Sequential(
            nn.ConvTranspose2d(in_channels=LATENT_VECTOR_SIZE, out_channels=GENER_FILTERS * 8,
                               kernel_size=4, stride=1, padding=0),
            nn.BatchNorm2d(GENER_FILTERS * 8),
            nn.ReLU(),
            nn.ConvTranspose2d(in_channels=GENER_FILTERS * 8, out_channels=GENER_FILTERS * 4,
                               kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(GENER_FILTERS * 4),
            nn.ReLU(),
            nn.ConvTranspose2d(in_channels=GENER_FILTERS * 4, out_channels=GENER_FILTERS * 2,
                               kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(GENER_FILTERS * 2),
            nn.ReLU(),
            nn.ConvTranspose2d(in_channels=GENER_FILTERS * 2, out_channels=GENER_FILTERS,
                               kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(GENER_FILTERS),
            nn.ReLU(),
            nn.ConvTranspose2d(in_channels=GENER_FILTERS, out_channels=output_shape[0],
                               kernel_size=4, stride=2, padding=1),
            nn.Tanh()
        )

    def forward(self, x):
        return self.pipe(x)

In [48]:
def iterate_batches(envs, batch_size = BATCH_SIZE):
    batch = [e.reset() for e in envs]
    env_gen = iter(lambda: random.choice(envs), None)
    
    while(True):
        e = next(env_gen) # get next env as random
        obs, reward, is_done, _ = e.step(e.action_space.sample())
        if(np.mean(obs) > 0.01):
            batch.append(obs)
        if len(batch) == batch_size:
            # normalize input!
            batch_np = np.array(batch, dtype=np.float32)
            batch_np = batch_np * (2.0 / 255.0 ) - 1.0
            yield torch.tensor(batch_np) # return batch as soon as it's made
            batch.clear() # do not end here -> make a endless generator
        if is_done:
            e.reset()
        

In [49]:
# main
device = torch.device("cuda")
envs = [
    InputWrapper(gym.make(name)) for name in ("Breakout-v0", "AirRaid-v0", "Pong-v0")
]

input_shape = envs[0].observation_space.shape

net_discr = Discriminator(input_shape=input_shape).to(device) # because it inherits NN.module
net_gener = Generator(output_shape=input_shape).to(device)

INFO: Making new env: Breakout-v0
INFO: Making new env: AirRaid-v0
INFO: Making new env: Pong-v0


In [56]:
objective = nn.BCELoss() # pixelwise loss
gen_optimizer = optim.Adam(
    params = net_gener.parameters(), lr = LEARNING_RATE,
    betas = (0.5, 0.999)) # the exponential decay params
dis_optimizer = optim.Adam(
    params = net_discr.parameters(), lr = LEARNING_RATE,
    betas = (0.5, 0.999))
writer =  SummaryWriter()


In [61]:
gen_losses = []
dis_losses = []
iter_no = 0

true_labels_v = torch.ones(BATCH_SIZE, device=device)
fake_labels_v = torch.zeros(BATCH_SIZE, device=device)

for batch_v in iterate_batches(envs):
    # gen input is 4d
    gen_input_v = torch.FloatTensor(BATCH_SIZE, LATENT_VECTOR_SIZE, 1,1)
    gen_input_v.normal_(0,1)
    gen_input_v = gen_input_v.to(device)
    batch_v = batch_v.to(device)
    gen_output_v = net_gener(gen_input_v)
    
    dis_optimizer.zero_grad()
    dis_output_true_v = net_discr(batch_v)
    dis_output_fake_v = net_discr(gen_output_v.detach()) # calling detach here stopps the gradient from flowing!
    
    dis_loss = objective(dis_output_fake_v, fake_labels_v) + objective(dis_output_true_v, true_labels_v)
    dis_loss.backward()
    dis_optimizer.step()
    dis_losses.append(dis_loss.item())  # call a value from 0d tensor
    
    
    gen_optimizer.zero_grad()
    dis_output_v = net_discr(gen_output_v) # no not call detach here because we need the gradients
    gen_loss_v = objective(dis_output_v, true_labels_v)
    gen_loss_v.backward()
    gen_optimizer.step()
    gen_losses.append(gen_loss_v.item())    
    
    iter_no += 1
    if iter_no % REPORT_EVERY_ITER == 0:
        log.info("Iter %d: gen_loss=%.3e, dis_loss=%.3e", iter_no, np.mean(gen_losses), np.mean(dis_losses))
        
        writer.add_scalar("gen_loss", np.mean(gen_losses), iter_no),
        writer.add_scalar("dis_loss", np.mean(dis_losses), iter_no)
        gen_losses = []
        dis_losses = [] # clear list
        
    if iter_no % SAVE_IMAGE_EVERY_ITER == 0:
        writer.add_image("fake", vutils.make_grid(
            gen_output_v.data[:64], normalize=True), iter_no)
        writer.add_image("real", vutils.make_grid(
            batch_v.data[:64], normalize=True), iter_no)

INFO: Iter 100: gen_loss=6.912e+00, dis_loss=8.432e-03
INFO: Iter 200: gen_loss=7.466e+00, dis_loss=2.898e-03
INFO: Iter 300: gen_loss=7.610e+00, dis_loss=1.723e-03
INFO: Iter 400: gen_loss=7.529e+00, dis_loss=1.398e-01
INFO: Iter 500: gen_loss=7.105e+00, dis_loss=9.364e-03
INFO: Iter 600: gen_loss=8.430e+00, dis_loss=4.214e-02
INFO: Iter 700: gen_loss=7.156e+00, dis_loss=1.220e-02
INFO: Iter 800: gen_loss=7.387e+00, dis_loss=3.086e-03
INFO: Iter 900: gen_loss=7.114e+00, dis_loss=3.681e-03
INFO: Iter 1000: gen_loss=7.358e+00, dis_loss=1.945e-03
INFO: Iter 1100: gen_loss=7.759e+00, dis_loss=1.433e-03
INFO: Iter 1200: gen_loss=9.129e+00, dis_loss=2.893e-02
INFO: Iter 1300: gen_loss=7.272e+00, dis_loss=7.217e-02
INFO: Iter 1400: gen_loss=5.934e+00, dis_loss=2.176e-02
INFO: Iter 1500: gen_loss=7.745e+00, dis_loss=1.376e-01
INFO: Iter 1600: gen_loss=5.647e+00, dis_loss=3.749e-01
INFO: Iter 1700: gen_loss=5.684e+00, dis_loss=3.101e-01
INFO: Iter 1800: gen_loss=4.140e+00, dis_loss=3.393e-01
I

KeyboardInterrupt: 

## PyTorch Ignite

In [63]:
from ignite.engine import Engine, Events
from ignite.metrics import RunningAverage
from ignite.contrib.handlers import tensorboard_logger as tb_logger

In [64]:
net_discr = Discriminator(input_shape=input_shape).to(device)
net_gener = Generator(output_shape=input_shape).to(device)
gen_optimizer = optim.Adam(params=net_gener.parameters(), lr=LEARNING_RATE, betas=(0.5, 0.999))
dis_optimizer = optim.Adam(params=net_discr.parameters(), lr=LEARNING_RATE, betas=(0.5, 0.999))

In [65]:
true_labels_v = torch.ones(BATCH_SIZE, device=device)
fake_labels_v = torch.zeros(BATCH_SIZE, device=device)

In [77]:
def process_batch(trainer, batch): # a function to input into engine
    gen_input_v = torch.FloatTensor(BATCH_SIZE, LATENT_VECTOR_SIZE, 1, 1)
    gen_input_v.normal_(0,1)
    gen_input_v = gen_input_v.to(device)
    
    batch_v = batch.to(device)
    
    gen_output_v = net_gener(gen_input_v)
    
    # train discriptor
    dis_optimizer.zero_grad()
    dis_output_true_v = net_discr(batch_v)
    dis_output_fake_v = net_discr(gen_output_v.detach())
    dis_loss = objective(dis_output_fake_v, fake_labels_v) + objective(dis_output_true_v, true_labels_v)
    dis_loss.backward()
    dis_optimizer.step()
    
    # train generator
    gen_optimizer.zero_grad()
    dis_output_v = net_discr(gen_output_v)
    gen_loss = objective(dis_output_v, true_labels_v)
    gen_loss.backward()
    gen_optimizer.step()
    
    if trainer.state.iteration % SAVE_IMAGE_EVERY_ITER == 0:
        fake_img = vutils.make_grid(gen_output_v.data[:64], normalize=True)
        trainer.tb.writer.add_image("fake", fake_img, trainer.state.iteration)
        real_img = vutils.make_grid(batch_v.data[:64], normalize=True)
        trainer.tb.writer.add_image("real", real_img, trainer.state.iteration)
        trainer.tb.writer.flush() # use flush!
        
    return dis_loss.item(), gen_loss.item() # return train results

In [78]:
engine = Engine(process_batch) # attach our train function here!
tb = tb_logger.TensorboardLogger(log_dir=None) # where?
engine.tb = tb # save module like this
RunningAverage(output_transform=lambda out: out[1]).attach(engine, "avg_loss_gen") # attach metrics -> how?
RunningAverage(output_transform=lambda out: out[0]).attach(engine, "avg_loss_dis")

handler = tb_logger.OutputHandler(tag="train", metric_names=["avg_loss_gen", "avg_loss_dis"])
tb.attach(engine, log_handler=handler, event_name=Events.ITERATION_COMPLETED) # attach logger

In [79]:
@engine.on(Events.ITERATION_COMPLETED)
def log_losses(trainer):
    if trainer.state.iteration % REPORT_EVERY_ITER == 0:
        log.info("%d: gen_loss=%f, dis_loss=%f",
                 trainer.state.iteration,
                 trainer.state.metrics["avg_loss_gen"],
                 trainer.state.metrics["avg_loss_dis"])

In [ ]:
engine.run(data=iterate_batches(envs))

INFO: 100: gen_loss=5.345626, dis_loss=0.113565
INFO: 200: gen_loss=6.632696, dis_loss=0.019313
INFO: 300: gen_loss=7.193804, dis_loss=0.005108
INFO: 400: gen_loss=7.378413, dis_loss=0.002513
INFO: 500: gen_loss=7.750945, dis_loss=0.001690
INFO: 600: gen_loss=8.026420, dis_loss=0.001059
INFO: 700: gen_loss=8.222263, dis_loss=0.001084
INFO: 800: gen_loss=8.064052, dis_loss=0.239881
INFO: 900: gen_loss=6.484239, dis_loss=0.048323
INFO: 1000: gen_loss=5.760964, dis_loss=0.087028
INFO: 1100: gen_loss=6.106640, dis_loss=0.020134
INFO: 1200: gen_loss=6.309183, dis_loss=0.007554
INFO: 1300: gen_loss=6.539118, dis_loss=0.004120
INFO: 1400: gen_loss=6.629929, dis_loss=0.003528
INFO: 1500: gen_loss=6.843645, dis_loss=0.003711
INFO: 1600: gen_loss=7.151535, dis_loss=0.003226
INFO: 1700: gen_loss=7.473411, dis_loss=0.001459
INFO: 1800: gen_loss=7.777237, dis_loss=0.001007
INFO: 1900: gen_loss=7.816696, dis_loss=0.000981
INFO: 2000: gen_loss=9.046327, dis_loss=0.043740
INFO: 2100: gen_loss=7.076248